In [ ]:
import sys
sys.path.extend([ '../', '../../', '../../..'])
import torch
import dataloader as dl
from args import Args
# from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from model_magnn import MAG_ConEn, MAG_NetEn, EdgePredictor, MAG_classify
from train2 import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.dblp()
torch.cuda.empty_cache()

In [ ]:
data = torch.load('../../data/dblp_data.pt')
print(data)

In [ ]:
print(data['p','walk','a'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['a']['x'] = data['a']['x'].to(device)
data['p']['x'] = data['p']['x'].to(device)
data['t']['x'] = data['t']['x'].to(device)

data['c_embed']['x'] = data['c_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device) 
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 't']['edge_index'] = data['a', 'walk', 't']['edge_index'].to(device)
data['a', 'walk', 'c']['edge_index'] = data['a', 'walk', 'c']['edge_index'].to(device)

data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 't']['edge_index'] = data['p', 'walk', 't']['edge_index'].to(device)
data['p', 'walk', 'c']['edge_index'] = data['p', 'walk', 'c']['edge_index'].to(device)

data['t', 'walk', 'a']['edge_index'] = data['t', 'walk', 'a']['edge_index'].to(device)
data['t', 'walk', 'p']['edge_index'] = data['t', 'walk', 'p']['edge_index'].to(device)
data['t', 'walk', 't']['edge_index'] = data['t', 'walk', 't']['edge_index'].to(device)
data['t', 'walk', 'c']['edge_index'] = data['t', 'walk', 'c']['edge_index'].to(device)

data['c', 'walk', 'a']['edge_index'] = data['c', 'walk', 'a']['edge_index'].to(device)
data['c', 'walk', 'p']['edge_index'] = data['c', 'walk', 'p']['edge_index'].to(device)
data['c', 'walk', 't']['edge_index'] = data['c', 'walk', 't']['edge_index'].to(device)
data['c', 'walk', 'c']['edge_index'] = data['c', 'walk', 'c']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 't'].edge_index, data['a', 'walk', 'c'].edge_index ]

In [ ]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

In [ ]:
## MAGNN

encoder1 = MAG_ConEn(args.embed_dim, args, args.dropout)
encoder2 = MAG_NetEn(args.embed_dim, args.dropout)
# encoder1 = torch.load('pretrained/encoder1.pth')
# encoder2 = torch.load('pretrained/encoder2.pth')
classifier = MAG_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_t = EdgePredictor(args.embed_dim)
decoder_c = EdgePredictor(args.embed_dim)
# decoder_a = torch.load('pretrained/decoder_a.pth')
# decoder_p = torch.load('pretrained/decoder_p.pth')
# decoder_t = torch.load('pretrained/decoder_t.pth')
# decoder_c = torch.load('pretrained/decoder_c.pth')

decoder_list = [decoder_a, decoder_p, decoder_t, decoder_c]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

In [ ]:
#HGNN 

# encoder1 = Het_ConEn(args.embed_dim, args, args.dropout)
# encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# # encoder1 = torch.load('pretrained/encoder1.pth')
# # encoder2 = torch.load('pretrained/encoder2.pth')
# classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
# decoder_a = EdgePredictor(args.embed_dim)
# decoder_p = EdgePredictor(args.embed_dim)
# decoder_t = EdgePredictor(args.embed_dim)
# decoder_c = EdgePredictor(args.embed_dim)
# # decoder_a = torch.load('pretrained/decoder_a.pth')
# # decoder_p = torch.load('pretrained/decoder_p.pth')
# # decoder_t = torch.load('pretrained/decoder_t.pth')
# # decoder_c = torch.load('pretrained/decoder_c.pth')

# decoder_list = [decoder_a, decoder_p, decoder_t, decoder_c]

# encoder1.to(device)
# encoder2.to(device)
# classifier.to(device)
# for decoder in decoder_list:
#     decoder.to(device)

## Training Part

In [ ]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'preO')

In [ ]:
# test_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

In [ ]:
# torch.save(decoder_list[0], '../pretrained_magnn/decoder_a.pth')
# torch.save(decoder_list[1], '../pretrained_magnn/decoder_p.pth')
# torch.save(decoder_list[2], '../pretrained_magnn/decoder_t.pth')
# torch.save(decoder_list[3], '../pretrained_magnn/decoder_c.pth')
# torch.save(encoder1, '../pretrained_magnn/encoder1.pth')
# torch.save(encoder2, '../pretrained_magnn/encoder2.pth')